# Natural Language Processing

El Natural Language Processing o NLP és el tractament de dades textuals escrites en alguna llengua concreta. En aquesta pràctica veurem dues de les seves aplicacions: la recuperació d'informació i la classificació amb Naive Bayes.

<b>Dades del Projecte Gutenberg</b>. Les dades amb les quals treballarem primer són textos de l'escriptor anglès del segle XIX Henry Rider Haggard.

In [1]:
import re
from glob import glob
from os.path import basename

## Abans de començar


**\+ No es poden modificar les definicions de les funcions donades, ni canviar els noms de les variables i paràmetres ja donats**


**\+ En les funcions, s'especifica què serà i de quin tipus cada un dels paràmetres, cal respectar-ho**
 

## 1. Recuperació d'informació


La recuperació d'informació es preocupa pel problema que apareix quan un usuari vol cercar un document textual en una base de dades. En aquesta pràctica us donem una col·lecció de textos literaris del Projecte Gutenberg (https://www.gutenberg.org/). Com trobaríeu un llibre que contingui les paraules "eye", "father", "work"? Penseu en el cercador de Google: quan feu una cerca, el sistema ha de:

1. Trobar documents que contenen les vostres paraules clau,

2. Mostrar-vos els resultats de la cerca amb un ordre de major a menor rellevància.

Sobre aquest esquema bàsic es poden fer moltes millores, com ara la cerca aproximada de text o la personalització de resultats. A més, hi pot haver moltes maneres d'ordenar els resultats. Aquí veurem una versió relativament simple del procés, i en particular només cercarem de manera exacta. Veient el següent esquema del procés de recuperació d'informació:

<img width="60%" src="img/esquema-recuperacio-informacio.png">

nosaltres farem les parts d'indexació, query, i sorting.

### Indexació

La indexació consisteix en incorporar documents al nostre sistema de manera que ens resulti senzill trobar-los posteriorment. Per efectuar tal tasca, emprarem una estructura de dades anomenada **Índex Invers**. Per entendre com es fabrica, considerem una taula de documents amb algunes de les paraules que contenen: cada columna representa un document, i cada fila ens indica si aquell document conté la corresponent paraula.

|          | Document 1 | Document 2 | Document 3 | Document 4 |
|:--------:|:----------:|:----------:|:----------:|:----------:|
| Einstein |  1         |  1         |  0         |  0         |
|  Hubble  |  1         |  1         |  1         |  0         |
|   Fermi  |  1         |  0         |  0         |  0         |
|  Winfrey |  0         |  0         |  0         |  1         |
|   Dylan  |  0         |  0         |  1         |  1         |

Quins documents parlen de Hubble? Llegint la segona fila està molt clar: els tres primers. L'essència de l'índex invers es basa en cercar els termes que volem al llistat de paraules indexades, i retornar la seva "fila", és a dir, els documents que la contenen.

A la pràctica, no podem guardar el nostre índex en una taula: tindríem un problema d'espai. A banda, necessitem una manera ràpida de cercar a la llista de paraules indexades. La cerca ràpida s'aconsegueix amb una estructura de tipus hash. Si volem una estructura de tipus hash que pugui guardar informació associada a cada terme, necessitem usar **diccionaris**. A cada terme li assignarem un conjunt (`set`) de strings, cadascuna amb l'identificador del document que pertoqui. D'aquesta manera, la taula anterior quedaria convertida en el següent diccionari:

```
{
    "Einstein": {"Document 1", "Document 2"},
    "Hubble": {"Document 1", "Document 2", "Document 3"},
    "Fermi": {"Document 1"},
    "Winfrey": {"Document 4"},
    "Dylan": {"Document 3", "Document 4"}
}
```

Per crear el vostre index invers, us proporcionem la funció `tokenize` i la classe `IndexInvers`. La funció `tokenize` és un generador que agafa un text i va treient cadascuna de les paraules que conté, de manera que podeu iterar sense haver de pensar com separar els espais i la puntuació. 

La classe `IndexInvers` només conté el constructor, en el qual es creen els seus atributs:
- `tdf` (Term-document frequency): el diccionari que acabem de descriure. És del tipus `defaultdict(set)`. Això vol dir que podem accedir (i tractar amb) qualsevol clau. Si per exemple tenim la paraula "Avui" i la clau `tdf["Avui"]` no existeix, en comptes de llençar una excepció, el `defaultdict` ens crearà la clau i aquesta contidrà un conjunt buit. 
- `doc_ids` (identificadors de document): llista on guardarem els identificadors dels documents que indexem. Serà necessari al moment de cercar a l'índex.

Els identificadors consistiran en les rutes als arxius de text indexats.

In [2]:
from collections import defaultdict


def tokenize(text, lowercase=True):
    text = text.lower() if lowercase else text
    for match in re.finditer(r"\w+(\.?\w+)*", text):
        yield match.group()

        
class IndexInvers:
    
    def __init__(self):
        self.tdf = defaultdict(set)
        self.doc_ids = []

**Exercici 1.** 

Implementeu la funció `index_document`, que a partir d'un índex invers, un identificador de document i un iterable de paraules, afegeixi el document a l'índex invers segons s'ha descrit al text introductori.

In [3]:
def index_document(inverse_index, doc_id, words):
    """Afegeix el document referenciat per doc_id i l'indexa d'acord amb les seves paraules. 
    Aquesta funció no ha de retornar res, només modificar els atributs de l'índex invers
    
    Input:
        inverse_index: objecte de tipus IndexInvers
        doc_id: string identificativa del document
        words: objecte iterable de les paraules del document
    """        
    # Iterate over all the words    
    for word in words:
        # Add the document ID to each word
        inverse_index.tdf[word].add(doc_id)
    # Add also on the docs_ids
    inverse_index.doc_ids.append(doc_id)

La funció `create_index` utilitza l'`index_document` que heu implementat per crear un índex invers amb els documents indicats. Executeu la següent cel·la per indexar els textos de la carpeta `haggard`. 

In [4]:
def create_index(filenames):
    """Crea i retorna un índex invers amb els documents d'una llista indexats.
        
    Input:
        filenames: llista de documents a indexar
        
    Returns:
        Un objecte de tipus IndexInvers
    """
    inverse_index = IndexInvers()
    
    for filename in filenames:
        index_document(inverse_index, 
                       basename(filename), 
                       tokenize(open(filename, encoding="utf-8").read()))    
    return inverse_index

inverse_index = create_index(glob('data/haggard/*.txt'))

Els següents tests haurien de donar `True` si heu implementat l'indexació correctament:

In [5]:
print('The Ghost Kings 8184.txt' in inverse_index.tdf['master'])
print('Cleopatra 2769.txt' in inverse_index.tdf['children'])

True
True


**Exercici 2.**

Amb l'índex creat, ja podem fer cerques! Implementeu la funció `query` de manera que, donada una paraula clau, ens retorni tots els documents que la contenen.

In [6]:
def query_one(inverse_index, term):
    """Troba tots els documents de l'índex que coincideixin amb el terme cercat.
    
    Input:
        inverse_index: objecte de tipus InverseIndex
        term: paraula 
        
    Returns:
        conjunt amb els documents que contenen la paraula
    """
    return inverse_index.tdf[term]   

In [7]:
print(query_one(inverse_index, "children"))
print(query_one(inverse_index, "father"))

{'Allan and the Ice Gods (1927) 0200201.txt', 'Cetywayo and his White Neighbours Remarks on Recent Events in Zululand, Natal, and the Transvaal 8667.txt', 'Mr. Meesons Will 11913.txt', 'Smith and the Pharaohs, and other Tales 6073.txt', 'A Yellow God an Idol of Africa 2857.txt', 'Moon of Israel 2856.txt', 'The People of the Mist 6769.txt', 'The Brethren 2762.txt', 'Lysbeth, a Tale of the Dutch 5754.txt', 'The Worlds Desire 2763.txt', 'Morning Star 2722.txt', 'Ayesha, the Return of She 5228.txt', 'King Solomons Mines 2166.txt', 'The Ghost Kings 8184.txt', 'Stories by English Authors Africa (Selected by Scribners) 1980.txt', 'Montezumas Daughter 1848.txt', 'Marie An Episode in The Life of the late Allan Quatermain 1690.txt', 'Elissa 2855.txt', 'The Wizard 2893.txt', 'Allan and the Holy Flower 5174.txt', 'She and Allan 5745.txt', 'Maiwas Revenge 2713.txt', 'Allans Wife 2727.txt', 'The Tale of Three Lions 2729.txt', 'Eric Brighteyes 2721.txt', 'Wisdoms Daughter (1923) 0200181.txt', 'Finish

La possibilitat de fer cerques ens és útil en el moment en què podem entrar més d'un terme simultàniament. Si cerquem el text "fa bon dia", volem trobar els documents que continguin aquestes tres paraules (no necessàriament en aquest ordre). Dit d'una altra manera, volem la intersecció de les cerques dels termes "fa", "bon" i "dia".

**Exercici 3.**

Implementeu la funció `query_terms` de manera que donada una llista de paraules clau `*terms` ens retorni tots els documents que continguin totes les paraules clau.

In [8]:
def query_terms(inverse_index, terms):
    """Troba tots els documents de l'índex que coincideixin amb els termes de la cerca.
    
    Input:
        inverse_index: objecte de tipus InverseIndex
        terms: llista de paraules
        
    Returns:
        conjunt amb els documents que coincideixen amb la cerca.
    """
    # Get the first term because we are gonna use the intersection function
    # This function will only get when the docs_id are the same
    result = inverse_index.tdf[terms[0]]
    for term in terms[1:]:
        result = result.intersection(inverse_index.tdf[term])
    return result

In [9]:
print(query_terms(inverse_index, ["eye", "father", "work", "today"]))

{'The Ivory Child 2841.txt', 'Allan Quatermain 711.txt', 'Finished 1724.txt', 'Ayesha, the Return of She 5228.txt', 'When the World Shook; being an account of the great adventure of Bastin, Bickley and Arbuthnot 1368.txt'}


**Exercici 4.**

Podem voler també cercar documents que continguin *alguna* de les paraules clau. Implementeu la funció `query_some_terms` per aconseguir aquest comportament.

In [10]:
def query_some_terms(inverse_index, terms):
    """Troba tots els documents de l'índex que coincideixin amb algun dels termes de la cerca.
    
    Input:
        inverse_index: objecte de tipus InverseIndexCounter
        terms: llista de paraules
        
    Returns:
        conjunt amb els documents que contenen alguna de les paraules
    """
    res = set()
    # For this function we will use union because we want any of the terms
    for term in terms:        
        res = res.union(inverse_index.tdf[term])
    return res

## Scoring

Amb la funció `query_terms`, ja podem cercar qualsevol document a la nostra base de dades. Però amb això no n'hi ha prou: què passarà quan la cerca retorni massa resultats? Necessitem una manera d'ordenar els documents que apareguin a la cerca, preferiblement de més a menys rellevància.

Per fer-ho, necessitem una versió modificada del nostre índex invers. Per començar, necessitem guardar la longitud de cada document. Com que cada document té un identificador, això ho podem fer directament amb un diccionari. També ho podem fer amb una estructura una mica diferent, anomenada `Counter` (https://docs.python.org/3/library/collections.html#collections.Counter).

Tal com diu la documentació de `collections`, un Counter és un diccionari que ens permet comptar. Cadascun dels seus valors és només un `int`, i podem incrementar-lo sense haver de definir cada clau. Té molta similitud amb un `defaultdict(int)`. Veiem-ho amb un exemple:

In [11]:
from collections import Counter

counter = Counter()
animals = ["gos", "gat", "gos", "gos", "gallina", "serp", "gat"]
for a in animals:
    counter[a] += 1
    
print(counter)

Counter({'gos': 3, 'gat': 2, 'gallina': 1, 'serp': 1})


L'avantatge de counter és que no cal fer el bucle for! El següent codi fa exactament el mateix:

In [12]:
counter2 = Counter(animals)
print(counter2)

Counter({'gos': 3, 'gat': 2, 'gallina': 1, 'serp': 1})


També disposem del mètode `most_common`, que ens dóna els n termes més comuns.

In [13]:
print(counter2.most_common(2))

[('gos', 3), ('gat', 2)]


La utilitat del `Counter` pot no estar massa clara per comptar la longitud de cada text. Hi ha una altra aplicació per la qual ens serà molt útil. Per poder ordenar els resultats de la nostra cerca, voldrem saber quantes vegades surt una paraula a cada text. Això ho podem aconseguir canviant el `tdf` del nostre índex invers (que era un `defaultdict(set)`) per un `defaultdict(Counter)`. 

D'aquesta manera no només sabrem quins documents contenen cada paraula, sino també la seva freqüència. Això justifica el nom TDF (Term-Document Frequency).

In [14]:
class IndexInversCounter:
    """El mateix que IndexInvers, però utilitzant defaultict(Counter)
    """
    
    def __init__(self):
        self.tdf = defaultdict(Counter)
        self.lengths = Counter()
        self.doc_ids = []

**Exercici 5.**

Implementeu la funció `index_document_counter`, que a partir d'un índex invers `IndexInversCounter`, un identificador de document i un iterable de paraules, afegeixi el document a l'índex invers segons s'ha descrit.

In [15]:
def index_document_counter(inverse_index, doc_id, words):
    """Afegeix el document referenciat per doc_id i l'indexa d'acord amb les seves paraules. 
    Aquesta funció no ha de retornar res, només modificar els atributs de l'índex invers
    
    Input:
        inverse_index: objecte de tipus IndexInversCounter
        doc_id: string identificativa del document
        words: objecte iterable de les paraules del document
    """
    # Create a counter from all the words
    counted_words = Counter(words)        
    # Iterate over all the words
    for word in counted_words:
        # For each word and each document, save the number of occurrence of that word
        inverse_index.tdf[word][doc_id] = counted_words[word]
    
    # Finally sum all the values from counter to have the length of the document
    inverse_index.lengths[doc_id] = sum(counted_words.values()) 
    
    # Also, append the document into the doc_ids
    inverse_index.doc_ids.append(doc_id)   

Executeu la següent cel·la per indexar els textos de la carpeta `haggard`, aquest cop amb comptatge de les longituds de cada text:

In [16]:
def create_counter_index(filenames):
    """Retorna un índex invers amb la llista d'arxius indexats.
    
    Input:
        filenames: llista d'arxius
        
    Returns:
        objecte del tipus IndexInversCounter

    """
    inverse_index = IndexInversCounter()

    for filename in filenames:
        index_document_counter(inverse_index, basename(filename), tokenize(open(filename, encoding="utf-8").read()))
    
    return inverse_index

inverse_index_counter = create_counter_index(glob('data/haggard/*.txt'))

**Exercici 6.**

Ara la cerca no funcionarà, ja que estem treballant amb `Counter`s en comptes de `set`s. Modifiqueu la funció `query_terms` perquè funcioni.

In [17]:
def query_terms_counter(inverse_index, terms):
    """Troba tots els documents de l'índex que coincideixin amb els termes de la cerca.
    
    Input:
        inverse_index: objecte de tipus InverseIndexCounter
        terms: llista de paraules
        
    Returns:
        conjunt amb els documents que coincideixen amb la cerca.
    """            
    # Get always the first term
    result = set(inverse_index.tdf[terms[0]])    
    # Intersect with the rest of terms
    for term in terms[1:]:
        result = result.intersection(set(inverse_index.tdf[term]))
    return result    

In [18]:
print(query_terms_counter(inverse_index_counter, ["eye", "father", "work"]))

{'Allan and the Ice Gods (1927) 0200201.txt', 'Cetywayo and his White Neighbours Remarks on Recent Events in Zululand, Natal, and the Transvaal 8667.txt', 'Mr. Meesons Will 11913.txt', 'Smith and the Pharaohs, and other Tales 6073.txt', 'A Yellow God an Idol of Africa 2857.txt', 'Moon of Israel 2856.txt', 'The People of the Mist 6769.txt', 'The Brethren 2762.txt', 'Lysbeth, a Tale of the Dutch 5754.txt', 'The Worlds Desire 2763.txt', 'Morning Star 2722.txt', 'Ayesha, the Return of She 5228.txt', 'King Solomons Mines 2166.txt', 'The Ghost Kings 8184.txt', 'Stories by English Authors Africa (Selected by Scribners) 1980.txt', 'Montezumas Daughter 1848.txt', 'Marie An Episode in The Life of the late Allan Quatermain 1690.txt', 'Elissa 2855.txt', 'The Wizard 2893.txt', 'Allan and the Holy Flower 5174.txt', 'She and Allan 5745.txt', 'Maiwas Revenge 2713.txt', 'Allans Wife 2727.txt', 'The Tale of Three Lions 2729.txt', 'Eric Brighteyes 2721.txt', 'Wisdoms Daughter (1923) 0200181.txt', 'Finish

Com hem esmentat, hem introduit la modificació amb `Counter` per poder tractar el problema del scoring. Hi ha diverses maneres d'ordenar els resultats de la cerca al nostre índex invers, ara en veurem dues. Quina és la manera més senzilla que se'ns ocorre per ordenar resultats?

Podem ordenar els documents segons en quin d'ells apareixen més vegades les paraules que hem cercat. És a dir, si cerquem "avui" i en un document hi surt la paraula 10 vegades, li donarem més pes (més puntuació) que a un document on hi surti 5 vegades. D'aquesta manera, la puntuació del document $D$ al cercar els termes $q_1,\dots,q_n$ serà
$$
    \mathrm{score}_D(q_1,\dots,d_n) = \sum_{i=1,\dots,n} freq(q_i,D).
$$
Aquesta manera de puntuar l'anomenarem "naive". 

**Exercici 7.**

Implementeu la funció de puntuació naive per un document donat.

In [19]:
def score_naive(inverse_index, doc_id, terms):
    """Puntuació naive d'un document per una cerca concreta
    
    Inputs:
        inverse_index: objecte de tipus IndexInversCounter
        doc_id: identificador del document a puntuar
        terms: llista de paraules
        
    Returns:
        un nombre amb la puntuació naive del document, segons la fórmula naive
    """
    # Start with a score as 0
    score = 0
    # Iterate over all the terms
    for term in terms:
        # Get the term and the document and sum the occurrence
        score += inverse_index.tdf[term][doc_id]
    return score

In [20]:
print(score_naive(inverse_index_counter, "The Ghost Kings 8184.txt", ["eye", "father", "work"]))
print(score_naive(inverse_index_counter, "The Virgin of the Sun 3153.txt", ["eye", "father", "work"]))


212
144


Ràpidament ens adonem que la manera naive de puntuar pot no ser la millor. Si un document és molt més llarg que la resta, és molt mes probable que la paraula que cerquem hi aparegui freqüentment. Això li donarà una puntuació molt major que a un document més curt que pot ser més rellevant. A banda, cal tenir en compte que certes paraules (com ara els connectors i les preposicions) són massa comuns per donar informació sobre la rellevància d'un resultat de cerca.

Una solució a aquests problemes ve donada pel mètode de puntuació Okapi BM25 (https://en.wikipedia.org/wiki/Okapi_BM25). Ve donat per la següent fórmula:

$$\operatorname{score}_D\left(q_{1}, q_{2}, \ldots, q_{n}\right)
=
\sum_{i=1}^{n} I D F\left(q_{i}\right) \cdot 
\frac{freq\left(q_{i}, D\right) \cdot\left(k_{1}+1\right)}
{freq\left(q_{i}, D\right)+k_{1} \cdot\left(1-b+b \cdot \frac{|D|}{\operatorname{avgdl}}\right)},$$
on hi tenim la següent informació:

- $freq(q_i, D)$ és la freqüència del terme $q_i$ al document $D$.
- $k_1=0.75$ i $b=1.2$ són paràmetres, que es poden ajustar per donar resultats diferents.
- $|D|$ és la longitud del document $D$.
- $avgdl$ és la longitud mitjana de tots els documents (*av*era*g*e *d*ocument *l*ength)

El terme $IDF(q_i)$ es coneix com la Inverse Document Frequency, donat per la fórmula
$$I D F\left(q_{i}\right)=\log\left( 1+ \frac{N-n\left(q_{i}\right)+0.5}{n\left(q_{i}\right)+0.5} \right),$$
on $N$ és el nombre de documents indexats, i $n(q_i)$ és el nombre de documents que contenen el terme $q_i$.

**Exercici 8.**

Implementeu la fórmula de la puntuació Okapi BM25

In [21]:
from math import log
import numpy as np
k1 = 0.75
b = 1.2

def score_okapi(inverse_index, doc_id, terms):
    """Puntuació Okapi BM25 d'un document per una cerca concreta
    
    Inputs:
        inverse_index: objecte de tipus IndexInversCounter
        doc_id: identificador del document a puntuar
        terms: llista de paraules
        
    Returns:
        un nombre amb la puntuació Okapi BM25 del document, segons la fórmula descrita.
    """
    # Start with a score as 0
    score = 0    
    # Iterate over all the terms
    for term in terms:
        # Get all the values to apply the formula
        N = len(inverse_index.doc_ids)
        nQ = len(inverse_index.tdf[term])
        idf = log(1 + (N - nQ + 0.5) /(nQ + 0.5))
        freq = inverse_index.tdf[term][doc_id]        
        doc_len = inverse_index.lengths[doc_id]
        mean = np.mean(list(inverse_index.lengths.values()))
        
        # Just apply the formula
        score += idf * (freq * (k1 + 1)) / (freq + k1 * (1 - b + b * (doc_len / mean )))        
    return score    

In [22]:
print(score_okapi(inverse_index_counter, "Hunter Quatermain's Story 2728.txt", ["eye", "father", "work"]))
print(score_okapi(inverse_index_counter, "The Virgin of the Sun 3153.txt", ["eye", "father", "work"]))
print(score_okapi(inverse_index_counter, "Nada the Lily 1207.txt", ["eye", "father", "work"]))


0.0
0.09786966124624634
0.09755473303338251


**Exercici 9.**

Implementeu la cerca de documents amb ordenació per puntuació. La funció `query_n` ha d'admetre un nombre arbitrari de paraules clau, un nombre de resultats màxim esperats, i una funció de puntuació a través del paràmetre `score`.

In [23]:
def query_n(inverse_index, terms, n=10, score=score_naive):
    """Cerca dels n documents amb millor puntuació.
    
    Inputs:
        inverse_index: objecte de tipus IndexInversCounter
        terms: llista de paraules
        n: número de documents a recuperar
        score: funció de puntuació a utilitzar
        
    Returns:
        llista dels n documents amb millor puntuació, ordenats de millor a pitjor.
    """
    # Get all the documents with these terms
    query = query_terms_counter(inverse_index, terms)
    # Sort the documents by the score
    result = sorted(query, key=lambda x: score(inverse_index, x, terms), reverse=True)
    # Just return the number of documents that we want
    return result[:n]

In [24]:
print(query_n(inverse_index_counter, ["eye", "father", "work"]))

['Nada the Lily 1207.txt', 'Dawn 10892.txt', 'Stella Fregelius 6051.txt', 'Regeneration 13434.txt', 'Benita, an African romance 2761.txt', 'Lysbeth, a Tale of the Dutch 5754.txt', 'Colonel Quaritch, V.C. A Tale of Country Life 11882.txt', 'Love Eternal 3709.txt', 'Montezumas Daughter 1848.txt', 'Marie An Episode in The Life of the late Allan Quatermain 1690.txt']


In [25]:
print(query_n(inverse_index_counter, ["eye", "father", "work"], score=score_okapi))

['Hunter Quatermains Story 2728.txt', 'The Tale of Three Lions 2729.txt', 'The Mahatma and the Hare 2764.txt', 'Maiwas Revenge 2713.txt', 'Allans Wife 2727.txt', 'King Solomons Mines 2166.txt', 'Allan and the Ice Gods (1927) 0200201.txt', 'Allan and the Holy Flower 5174.txt', 'Dawn 10892.txt', 'Lysbeth, a Tale of the Dutch 5754.txt']
